In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
df = pd.read_csv("../datasets/winequality.csv", sep=";")
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [15]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df["quality"], test_size=0.2, random_state=42, stratify=df["quality"])

x_train.shape, x_test.shape

((3918, 11), (980, 11))

In [28]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(max_iter=500, hidden_layer_sizes=(16, 8, 8), random_state=42).fit(x_train, y_train)

In [32]:
# baseline accuracy score
mlp.score(x_test, y_test)

0.5173469387755102

## Sequential Forward selection

In [36]:
from sklearn.neural_network import MLPClassifier

# all input features
all_features = set(df.columns) - {"quality"}

feature_selection = []
prev_acc = 0

# iterate over the size of all features
for _ in all_features:
    max_acc = []
    # iterate over the features not yet included in the selection
    for x in all_features-set(feature_selection):
        # train MLP on current selection plus feature x and evaluate its score
        mlp = MLPClassifier(max_iter=500, hidden_layer_sizes=(16, 8, 8), random_state=42).fit(x_train[feature_selection+[x]], y_train)
        acc = mlp.score(x_test[feature_selection+[x]], y_test)
        max_acc.append((acc, x))

    # sort so that first element is the feature that together with
    # selection produced the highest accuracy
    max_acc.sort(key=lambda t: t[0], reverse=True)
    # if the feature with the highest accuracy did not increase the previous accuracy, then quit
    if max_acc[0][0] < prev_acc: break
    # append feature with highest accuracy 
    feature_selection.append(max_acc[0][1])
    prev_acc = max_acc[0][0]
    print(max_acc)

print(feature_selection)

[(0.5214285714285715, 'alcohol'), (0.4663265306122449, 'total sulfur dioxide'), (0.45, 'free sulfur dioxide'), (0.4489795918367347, 'chlorides'), (0.4489795918367347, 'density'), (0.4489795918367347, 'pH'), (0.4489795918367347, 'sulphates'), (0.4448979591836735, 'citric acid'), (0.44081632653061226, 'residual sugar'), (0.44081632653061226, 'fixed acidity'), (0.43673469387755104, 'volatile acidity')]
[(0.5265306122448979, 'sulphates'), (0.5255102040816326, 'citric acid'), (0.5224489795918368, 'volatile acidity'), (0.5204081632653061, 'fixed acidity'), (0.5163265306122449, 'density'), (0.513265306122449, 'free sulfur dioxide'), (0.5091836734693878, 'chlorides'), (0.49795918367346936, 'pH'), (0.4959183673469388, 'total sulfur dioxide'), (0.49081632653061225, 'residual sugar')]
[(0.5295918367346939, 'citric acid'), (0.5285714285714286, 'density'), (0.5244897959183673, 'volatile acidity'), (0.523469387755102, 'chlorides'), (0.5204081632653061, 'free sulfur dioxide'), (0.5163265306122449, 'p

In [37]:
# feature set with only ['alcohol', 'sulphates', 'citric acid'] has a similar score than with all 11 features
mlp = MLPClassifier(max_iter=500, hidden_layer_sizes=(16, 8, 8), random_state=42).fit(x_train[feature_selection], y_train)
mlp.score(x_test[feature_selection], y_test)

0.5295918367346939

## Sequential backward selection

In [43]:
from sklearn.neural_network import MLPClassifier

# all input features
all_features = set(df.columns) - {"quality"}

# start with every feature
feature_selection = [f for f in all_features]
prev_acc = 0

# iterate over the size of all features
for _ in all_features:
    max_acc = []
    # iterate over the selected features
    for x in feature_selection:
        # train MLP on current selection minus feature x and evaluate its score
        features = list(set(feature_selection)-{x})
        mlp = MLPClassifier(max_iter=500, hidden_layer_sizes=(16, 8, 8), random_state=42).fit(x_train[features], y_train)
        acc = mlp.score(x_test[features], y_test)
        max_acc.append((acc, x))

    # sort so that first element is the feature that together with
    # selection produced the highest accuracy
    max_acc.sort(key=lambda t: t[0], reverse=True)
    # if the feature with the highest accuracy did not increase the previous accuracy, then quit
    if abs(max_acc[0][0] - prev_acc) > 0.05: continue 
    # append feature with highest accuracy 
    feature_selection.remove(max_acc[0][1])
    prev_acc = max_acc[0][0]
    print(max_acc)

print(feature_selection)

/usr/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [41]:
mlp = MLPClassifier(max_iter=500, hidden_layer_sizes=(16, 8, 8), random_state=42).fit(x_train[feature_selection], y_train)
mlp.score(x_test[feature_selection], y_test)

0.539795918367347